In [122]:
import random 
import numpy as np
import torch


In [123]:
def Loss(Y,beta,X,lamd,p,K):
    sum=0
    
    for k in range(K):
        sum_betaX=0

        for j in range(p):
            sum_betaX=sum_betaX+beta[j,k]*X[j,k,:]

        sum=sum+0.5*torch.norm(Y[k,:].flatten()-sum_betaX.flatten())**2
    
    norm_infinite,indice=torch.max(torch.abs(beta),1)
    sum=sum+lamd*torch.sum(norm_infinite)

    return sum


In [124]:
#initialize settings
p=10
K=5
n_k=60
lamd=0

X=torch.randn(p,K,n_k)
for j in range(p):
    for k in range(K):
        X[j,k,:]=X[j,k,:]/torch.norm(X[j,k,:])

beta_true=torch.randint(low=-5,high=5,size=[p,K])
# print(beta_true)
Y=torch.zeros(K,n_k)

for k in range(K):
    sum=0
    for j in range(p):
        sum=sum+beta_true[j,k]*X[j,k,:]
    Y[k,:]=sum+torch.tensor(np.random.normal(loc=0,scale=0,size=X[j,k,:].size())).float()

# print(Y.size())


In [125]:
#C_ij
C=torch.zeros(p,K)
for j in range(p):
    for k in range(K):
        C[j,k]=torch.matmul(X[j,k,:].flatten(),Y[k,:].flatten())
        # print(C[j,k])
#D_ij^k
D=torch.zeros(p,p,K)
for k in range(K):
    for i in range(p):
        for j in range(p):
            D[i,j,k]=torch.matmul(X[i,k,:].flatten(),X[j,k,:].flatten())


# beta=torch.zeros(size=beta_true.size())
# print(C.size())
# print(D.size())

# print(D[:,:,1])


In [126]:
# beta=beta_true+torch.tensor(np.random.normal(loc=0,scale=9,size=beta_true.size())).float()
beta=torch.tensor(np.random.normal(loc=0,scale=25,size=beta_true.size())).float()
loss=Loss(Y,beta,X,lamd,p,K)
print(loss)

tensor(18844.9473)


In [127]:
def single_blcok(beta,p,K,lamd,X,Y,C,D,j_round):
    
    alpha=beta
    for k in range(K):
        beta[j_round,k]=C[j_round,k]-torch.matmul(beta[:,k].flatten().float(),D[:,j_round,k].flatten().float())+beta[j_round,k]*D[j_round,j_round,k]

    if (lamd!=0):
        if (torch.sum(torch.abs(alpha[j_round,:]))<=lamd):
            beta[j_round,:]=0
    
        alpha_j=alpha[j_round,:]
        alpha_j_abs=torch.abs(alpha_j)
        alpha_sorted,index1=torch.sort(alpha_j_abs,descending=True)
        alpha_j_sortred=torch.index_select(alpha_j,0,index1)
        judge=torch.zeros(size=alpha_j_abs.size())
    
        for l in range(K):
            judge[l]=(torch.sum(alpha_j_abs[0:l+1])-lamd)/(l+1)
    
        max_index=torch.argmax(judge)
    
    
        for k in range(max_index):
            beta[j_round,k]=torch.sign(alpha_j_sortred[k])/(max_index+1)*(torch.sum(alpha_j_abs[0:max_index+1])-lamd)

        for k in range(max_index,K):
            beta[j_round,k]=alpha_j_sortred[k]

        b,index2=torch.sort(index1)
    
        beta[j_round,:]=torch.index_select(beta[j_round,:],0,index2)
   
    return beta

In [128]:
delta_loss=100
while(delta_loss>=0.001):
    for j_round in range(p):
        beta=single_blcok(beta,p,K,lamd,X,Y,C,D,j_round)
    loss_new=Loss(Y,beta,X,lamd,p,K)
    delta_loss=torch.abs(loss_new-loss)
    loss=loss_new
    print(Loss(Y,beta,X,lamd,p,K))

print("%f",beta)
print("%f",beta_true)

tensor(1391.7717)
tensor(78.2791)
tensor(4.3912)
tensor(0.3057)
tensor(0.0297)
tensor(0.0037)
tensor(0.0005)
tensor(8.0211e-05)
%f tensor([[ 2.0005e+00,  3.9999e+00, -4.0004e+00, -5.0070e+00, -3.9987e+00],
        [ 2.0008e+00,  8.1062e-05, -4.9993e+00, -4.9949e+00, -4.0024e+00],
        [-4.1127e-05, -5.0000e+00, -2.0001e+00, -1.0045e+00,  4.0044e+00],
        [-4.9996e+00,  2.0000e+00,  1.0000e+00, -1.9881e+00, -1.9967e+00],
        [-4.9999e+00, -4.0000e+00, -1.3995e-04, -2.0035e+00, -2.8131e-03],
        [-5.0003e+00, -5.0000e+00,  1.0002e+00, -3.9946e+00, -4.9993e+00],
        [-1.0001e+00, -3.0000e+00, -5.0002e+00, -1.8835e-03,  4.0008e+00],
        [ 2.0002e+00, -4.0000e+00,  1.0001e+00,  3.0033e+00,  9.9798e-01],
        [-4.9999e+00,  4.0000e+00,  1.9998e+00,  2.0042e+00, -2.9993e+00],
        [-3.9998e+00, -1.0000e+00,  1.9998e+00, -2.0017e+00, -3.0019e+00]])
%f tensor([[ 2,  4, -4, -5, -4],
        [ 2,  0, -5, -5, -4],
        [ 0, -5, -2, -1,  4],
        [-5,  2,  1, -2, 

In [129]:

# alpha=beta
# for j in range(p):
#     for k in range(K):
#         alpha[j,k]=C[j,k]-torch.matmul(beta[:,k].flatten().float(),D[:,j,k].flatten().float())+beta[j,k]*D[j,j,k]
# print (alpha)
# print(Loss(Y,alpha,X,lamd,p,K))

In [130]:
# for j in range(p):
    
#     if (torch.sum(torch.abs(alpha[j,:]))<=lamd):
#         beta[j,:]=0
    
#     alpha_j=alpha[j,:]
#     alpha_j_abs=torch.abs(alpha_j)
#     alpha_sorted,index1=torch.sort(alpha_j_abs,descending=True)
#     alpha_j_sortred=torch.index_select(alpha_j,0,index1)
#     judge=torch.zeros(size=alpha_j_abs.size())
    
#     for l in range(K):
#         judge[l]=(torch.sum(alpha_j_abs[0:l+1])-lamd)/(l+1)
    
#     max_index=torch.argmax(judge)
    
    
#     for k in range(max_index):
#         beta[j,k]=torch.sign(alpha_j_sortred[k])/(max_index+1)*(torch.sum(alpha_j_abs[0:max_index+1])-lamd)

#     for k in range(max_index,K):
#         beta[j,k]=alpha_j_sortred[k]

#     b,index2=torch.sort(index1)
    
#     beta[j,:]=torch.index_select(beta[j,:],0,index2)
    
# print(beta)


In [131]:
loss=Loss(Y,beta,X,lamd,p,K)
print(loss)


tensor(8.0211e-05)
